In [1]:
import time
from queue import PriorityQueue

class Node:
    def __init__(self, state, parent=None, move=None, depth=0, path_cost=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.path_cost = path_cost

    def __lt__(self, other):
        return self.path_cost < other.path_cost

    def get_possible_moves(self):
        possible_moves = []
        x, y = None, None
        for i in range(3):
            for j in range(3):
                if self.state[i][j] == 0:
                    x, y = i, j
        directions = [('up', (x-1, y)), ('down', (x+1, y)), ('left', (x, y-1)), ('right', (x, y+1))]
        for direction, (i, j) in directions:
            if 0 <= i < 3 and 0 <= j < 3:
                possible_moves.append(direction)
        return possible_moves

    def generate_child(self, move):
        x, y = None, None
        for i in range(3):
            for j in range(3):
                if self.state[i][j] == 0:
                    x, y = i, j
        new_state = [row.copy() for row in self.state]
        if move == 'up':
            new_state[x][y], new_state[x-1][y] = new_state[x-1][y], new_state[x][y]
        elif move == 'down':
            new_state[x][y], new_state[x+1][y] = new_state[x+1][y], new_state[x][y]
        elif move == 'left':
            new_state[x][y], new_state[x][y-1] = new_state[x][y-1], new_state[x][y]
        elif move == 'right':
            new_state[x][y], new_state[x][y+1] = new_state[x][y+1], new_state[x][y]
        return Node(new_state, self, move, self.depth + 1, self.path_cost + 1)

def h1(state, goal):
    return 0

def h2(state, goal):
    return sum(1 for i in range(3) for j in range(3) if state[i][j] and state[i][j] != goal[i][j])

def h3(state, goal):
    return sum(abs(i - x) + abs(j - y) for i in range(3) for j in range(3) for x in range(3) for y in range(3) if state[i][j] == goal[x][y])

def h4(state, goal):
    return h3(state, goal) + 1

def a_star_search(initial, goal, heuristic):
    explored = set()
    expanded_nodes_count = 0
    start_node = Node(initial, path_cost=0)
    frontier = PriorityQueue()
    frontier.put((heuristic(initial, goal), start_node))
    while not frontier.empty():
        _, current = frontier.get()
        explored.add(tuple(map(tuple, current.state)))
        if current.state == goal:
            return current, expanded_nodes_count, explored
        for move in current.get_possible_moves():
            child = current.generate_child(move)
            expanded_nodes_count += 1
            if tuple(map(tuple, child.state)) not in explored:
                f = child.path_cost + heuristic(child.state, goal)
                frontier.put((f, child))
    return None, expanded_nodes_count, explored

def is_solvable(state):
    flat_list = [num for sublist in state for num in sublist if num != 0]
    inv_count = 0
    for i in range(len(flat_list)):
        for j in range(i+1, len(flat_list)):
            if flat_list[i] > flat_list[j]:
                inv_count += 1
    return inv_count % 2 == 0

def check_monotonicity(heuristic):
    for node in expanded_states_by_heuristic[heuristic.__name__]:
        n = Node(list(map(list, node)))
        for move in n.get_possible_moves():
            child = n.generate_child(move)
            if heuristic(n.state, goal) > child.path_cost + heuristic(child.state, goal):
                return False
    return True

initial = [
    [1, 2, 3],
    [5, 0, 6],
    [7, 8, 4]
]
goal = [
    [1, 2, 3],
    [5, 8, 6],
    [0, 7, 4]
]

if not is_solvable(initial):
    print("The given puzzle is not solvable!")
else:
    expanded_states_by_heuristic = {}
    for heuristic in [h1, h2, h3, h4]:
        print(f"Using heuristic {heuristic.__name__}:")
        start_time = time.time()
        result, expanded_nodes, explored_states = a_star_search(initial, goal, heuristic)
        end_time = time.time()
        expanded_states_by_heuristic[heuristic.__name__] = explored_states

        if result:
            print(f"Solution found with depth: {result.depth}")
            print(f"Number of expanded nodes: {expanded_nodes}")
            print(f"Time taken: {end_time - start_time:.4f} seconds\n")
        else:
            print("No solution found!")

    for heuristic in [h2, h3]:
        is_monotonic = check_monotonicity(heuristic)
        print(f"Monotonicity for {heuristic.__name__}: {'Followed' if is_monotonic else 'Not Followed'}")


Using heuristic h1:
Solution found with depth: 2
Number of expanded nodes: 20
Time taken: 0.0003 seconds

Using heuristic h2:
Solution found with depth: 2
Number of expanded nodes: 7
Time taken: 0.0009 seconds

Using heuristic h3:
Solution found with depth: 2
Number of expanded nodes: 7
Time taken: 0.0004 seconds

Using heuristic h4:
Solution found with depth: 2
Number of expanded nodes: 7
Time taken: 0.0003 seconds

Monotonicity for h2: Followed
Monotonicity for h3: Not Followed
